In [1]:
import pandas as pd
import numpy as np

In [141]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel, FactorRange


from bokeh.palettes import Category20_16

output_notebook()

Loading BokehJS ...

# Reading in Data

In [142]:
pwd

u'/home/kirubel/Networking/Webranking/bokeh_app/scripts'

In [215]:
# load in params and websites
webs_paras = pd.read_csv('../data/normalized.csv')
webs = webs_paras.drop(['(Doc complete) Byets in','(Fully loaded) Bytes in','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests'], axis = 1)
webs_requests = webs_paras[['Site name','(Fully loaded) Requests','(Fully loaded) Requests','(Doc complete) Requests']]



 #webs['(Doc complete) Byets in'].describe()

list_of_params = list(webs.columns[1:].unique())

list_of_params.sort()

#len(list_of_params) 

#for i in range(49):
    #print i%10

# Function to Make Dataset for plot


In [253]:
def make_dataset(params_list, range_start = 0.0, range_end = 1, bin_width = 0.005):
    
    #check to make sure the start is less than the end
    assert range_start < range_end, "Start must be less than end!"
    
    #by_params = pd.DataFrame(columns=[ ,'Max', 'Avarage', 'Min','color'])
    by_params = pd.DataFrame(columns=[ 'left','right', 'proportion', 'p_proportion','p_interval', 'name', 'color']) 
   # 
    
    range_extent = range_end - range_start
    values = ['Min', "Avarage", 'Max']
    # Iterate through all the parameters 
    for i, para_name in enumerate(params_list):
        
        #print para_name
        # Subset to the parameter
        subset = webs[para_name]
        
        # note: subset have to be a list of values
        
        # [webs.columns[i%6]]
        
        # Create a histogram with specified bins and range
        arr_hist, edges = np.histogram(subset,  
                                      bins = int(range_extent / bin_width),
                                      range = [range_start, range_end])

        # Divide the counts by the total to get a proportion and create df
        arr_df= pd.DataFrame({'proportion': arr_hist ,
                              'left': edges[:-1], 'right': edges[1:]}) #/ np.sum(arr_hist)
      
        # Format the proportion
        arr_df['p_proportion'] = ['%0.00005f' % proportion for proportion in arr_df['proportion']]
        
        # Format the interval
        arr_df['p_interval'] = ['%d to %d scale' % (left, right) for left, 
                               right in zip(arr_df['left'], arr_df['right'])]
        
        # Assign the parameter for labels
        arr_df['name'] = para_name
        
        # Color each parametr differently
        arr_df['color'] = Category20_16[i%16]
        
        # Add to the overall dataframe
        by_params = by_params.append(arr_df)
        
    # Overall dataframe
    by_params = by_params.sort_values(['name','left'])
    
    
    
    return by_params
        
        
        
        

In [254]:
make_dataset(list_of_params).head()

,color,left,name,p_interval,p_proportion,proportion,right
0,#1f77b4,0.000,(Fully loaded) Time,0 to 0 scale,1.00000,1,0.005
1,#1f77b4,0.005,(Fully loaded) Time,0 to 0 scale,0.00000,0,0.010
2,#1f77b4,0.010,(Fully loaded) Time,0 to 0 scale,0.00000,0,0.015
3,#1f77b4,0.015,(Fully loaded) Time,0 to 0 scale,1.00000,1,0.020
4,#1f77b4,0.020,(Fully loaded) Time,0 to 0 scale,2.00000,2,0.025


# Function to Make Plot from Data Source

In [255]:
def style(p):
    # Title
    p.title.align = 'center'
    p.title.text_font_size ='20pt'
    p.title.text_font = 'serif'
    
    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'
    
    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    return p

In [260]:
def make_plot(src):
    # Blank plot with correct labels
    p = figure(plot_width = 700, plot_height = 700,
              title = "Histogram of Parametes for the websites",
              x_axis_label = 'x_label', y_axis_label = "y_labe")
    
    # Quad glyphs to create a histogram
    p.quad(source=src, bottom =0,left = 'left', right = 'right', color ='color', top= 'proportion',fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
           hover_fill_alpha = 1.0, line_color = 'white') #top='proportion',
    
    # Hover tool with vline mode
    hover = HoverTool(tooltips=[('Parameter','@name'),
                               ('Difference','@p_interval'),
                                ('Proportion','p_proportion')
                               ],
                     mode='vline')
    
    p.add_tools(hover)
    # Stypling
    p = style(p)
    return p

In [261]:

p=make_plot(ColumnDataSource(make_dataset(list_of_params)))
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: proportion [renderer: GlyphRenderer(id='3335f533-322b-4f44-a789-728f19e7e4bc', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: left, proportion, right [renderer: GlyphRenderer(id='bede5682-a799-4bc0-8eb9-34a4134ff31a', ...)]
